In [1]:
from glob import glob
from random import sample
import json
from Levenshtein import distance as levenshtein_distance
import re

import texte


path = "Corpus/Mazarinades/*/*.xml"

files = glob(path)

print(len(files))

sample_mode = True

if sample_mode:
    files = sample(files, 200)
#     collection_textes = [texte.Texte(file) for file in tqdm(files)]
# else:
collection_textes = [e for e in texte.corpora(files) if e.plain]

In [3]:
with open("LGERM.json", "r") as f:
    lexique = json.load(f)

lexique = set(e.lower() for e in lexique)

chriffre_romain = re.compile("[IVXLCDM]+\.?")

def corrige_mot(mot):
    if chriffre_romain.fullmatch(mot):
        return (mot, "chiffre romain")

    if mot == "&amp;":
        return ("&", "ampersand")

    if mot.lower() in lexique:
        return (mot, "lexique")
    else:
        return (min(lexique, key=lambda x: levenshtein_distance(x, mot.lower())), "levenshtein")

with open("extremes/lexicalite_min.txt", "r") as f:
    hapaxmax = f.read()

for mot in hapaxmax.split():
    correction, type_ = corrige_mot(mot)
    if type_ != "lexique":
        print(type_, mot, correction)


levenshtein GALLICINIVM glucinivm
levenshtein NVPER neper
levenshtein AVDITIVM avditive
levenshtein FRANCIAM francia
levenshtein EXPILANTE espilante
levenshtein MAZARINO mazarin
levenshtein Gallo gall
levenshtein intelligentiam intelligentsia
levenshtein ? ô
levenshtein Iob. iobs
levenshtein 38. .l.
levenshtein praeparat preparat
levenshtein eſcam licam
levenshtein ſuam. guam
levenshtein Ibid. ibid
levenshtein PARIS, paris
chiffre romain M. M.
chiffre romain DC. DC.
chiffre romain XLIX. XLIX.
levenshtein GALLICINIVM glucinivm
levenshtein NVPER neper
levenshtein AVDITIVM avditive
levenshtein Franciams franciums
levenshtein expilate pilate
levenshtein Mazarino. mazarin
chiffre romain V V
levenshtein etaremRegine etayeroint
levenshtein matriper matricer
levenshtein ſe¬ hem
levenshtein licet: licet
levenshtein Surglre surgele
levenshtein prneipsRegpi¬ preimpregne
levenshtein claruit, claquiés
levenshtein Quemadmodum quemandés
levenshtein anſer anter
levenshtein animoſus animoies
levenshtei